In [7]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import os

# import customized modules
from models import DeepCon, DeepCov, BottleNeck, BasicBlock
from data import Psicov150

In [8]:
def get_output(x, length):
    p = x[0, 0:length, 0:length]
    p1 = (p + p.transpose(1, 0)) / 2
    p1 = p1.reshape(1, length, length)
    return p1

In [9]:
def predict(model, device, test_loader, pred_folder):
    model.eval()
    target_count = 0
    
    with torch.no_grad():    
        for data in test_loader:
            target_name = data['target'][0]
            sequence = data['sequence'][0]
            print('Predict {}/{} - {}'.format(target_count + 1, 
                                              len(test_loader.dataset), 
                                              target_name))
            cov = data['cov']
            cov = cov[0].to(device)
            pred = model(cov)
            pred = pred[0]
            # print('pred \n', pred)
            L = pred.shape[1]
            p = get_output(pred, L)[0]
            
            # write prediction to file
            rr = open(os.path.join(pred_folder, target_name + '.rr'), 'w')
            rr.write(sequence + "\n")
            
            for i in range(0, L):
                for j in range(i, L):
                    if abs(i - j) < 5:
                        continue
                    rr.write("%i %i 0 8 %.5f\n" %(i+1, j+1, p[i][j]))
            rr.close()

            target_count += 1

In [10]:
# load test target list

test_target_file = 'data/DeepCov/test/psicov150/target.lst.sorted'
test_target_list = []

# set folder for storing prediction files
pred_folder = 'data/DeepCov/test/psicov150/rr/'

with open(test_target_file) as target_file:
    for line in target_file:
        target = line.rstrip()
        test_target_list.append(target)   

print('Total test targets = {}'.format(len(test_target_list)))

Total test targets = 150


In [11]:
# load test data

test_dataset = Psicov150('data/DeepCov/test/psicov150/', test_target_list)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=0, shuffle=False)

In [12]:
# set device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# init model
model_name = 'DeepCon'

if model_name == 'DeepCov':
    net_args = {
        "block": BasicBlock,
        "layers": [10]
    }
    model = DeepCov(**net_args)

else: 
    model_name = 'DeepCon'
    net_args = {
        "block": BottleNeck,
        "layers": [8, 8]
    }
    model = DeepCon(**net_args)

try:
    # load checkpoint
    checkpoint = torch.load(model_name+'_checkpoint.pth.tar', map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    print('Pretrained model sucessfully loaded.')
except:
    print('Model loading failed. Stop prediction.')
    exit()
else:
    model.to(device)
    predict(model, device, test_loader, pred_folder)
    print('Prediction done.')

Pretrained model sucessfully loaded.
Predict 1/150 - 1a3aA
Predict 2/150 - 1a6mA
Predict 3/150 - 1a70A
Predict 4/150 - 1aapA
Predict 5/150 - 1abaA
Predict 6/150 - 1ag6A
Predict 7/150 - 1aoeA
Predict 8/150 - 1atlA
Predict 9/150 - 1atzA
Predict 10/150 - 1avsA
Predict 11/150 - 1bdoA
Predict 12/150 - 1bebA
Predict 13/150 - 1behA
Predict 14/150 - 1bkrA
Predict 15/150 - 1brfA
Predict 16/150 - 1bsgA
Predict 17/150 - 1c44A
Predict 18/150 - 1c52A
Predict 19/150 - 1c9oA
Predict 20/150 - 1cc8A
Predict 21/150 - 1chdA
Predict 22/150 - 1cjwA
Predict 23/150 - 1ckeA
Predict 24/150 - 1ctfA
Predict 25/150 - 1cxyA
Predict 26/150 - 1cznA
Predict 27/150 - 1d0qA
Predict 28/150 - 1d1qA
Predict 29/150 - 1d4oA
Predict 30/150 - 1dbxA
Predict 31/150 - 1dixA
Predict 32/150 - 1dlwA
Predict 33/150 - 1dmgA
Predict 34/150 - 1dqgA
Predict 35/150 - 1dsxA
Predict 36/150 - 1eazA
Predict 37/150 - 1ej0A
Predict 38/150 - 1ej8A
Predict 39/150 - 1ek0A
Predict 40/150 - 1f6bA
Predict 41/150 - 1fcyA
Predict 42/150 - 1fk5A
Predic